In [6]:
using CategoricalArrays, DataFrames, CSV, GLM


df = download("https://raw.githubusercontent.com/PharmCat/edu/main/csv/be.csv") |> CSV.File |> DataFrame

transform!(df, :Subject => categorical, renamecols=false)
transform!(df, :Period => categorical, renamecols=false)
transform!(df, :Sequence => categorical, renamecols=false)
transform!(df, :Formulation => categorical, renamecols=false)

#

,Subject,Sequence,Period,Formulation,Cmax
,Cat…,Cat…,Cat…,Cat…,Float64
1,3,TR,1,T,225.95
2,1,RT,1,R,181.09
3,2,RT,1,R,114.48
4,4,RT,1,R,176.91
5,5,TR,1,T,147.01
6,6,TR,1,T,97.53
7,7,RT,1,R,146.6
8,8,TR,1,T,45.58
9,9,RT,1,R,109.2


In [13]:
olscmax_null  = lm(@formula(log(Cmax) ~ Subject), df)
olscmax_form  = lm(@formula(log(Cmax) ~ Subject + Formulation), df)
olscmax_seq   = lm(@formula(log(Cmax) ~ Subject + Sequence), df)

olscmax  = lm(@formula(log(Cmax) ~ Sequence+Period+Formulation+Subject), df)

olscmax_noseq  = lm(@formula(log(Cmax) ~ Period+Formulation+Subject), df)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

:(log(Cmax)) ~ 1 + Period + Formulation + Subject

Coefficients:
──────────────────────────────────────────────────────────────────────────────────
                     Coef.  Std. Error       t  Pr(>|t|)    Lower 95%    Upper 95%
──────────────────────────────────────────────────────────────────────────────────
(Intercept)      5.26309     0.0596091   88.29    <1e-22   5.13672      5.38945
Period: 2        0.0709849   0.026658     2.66    0.0170   0.0144724    0.127497
Formulation: T  -0.0503868   0.026658    -1.89    0.0770  -0.106899     0.00612568
Subject: 2      -0.60704     0.079974    -7.59    <1e-05  -0.776578    -0.437503
Subject: 3       0.179358    0.079974     2.24    0.0394   0.00982058   0.348895
Subject: 4      -0.0709459   0.079974    -0.89    0.3882  -0.240483     0.0985915
Subject: 5      -0.3

In [ ]:
ftest(olscmax_null.model, olscmax_form.model, olscmax.model)

In [8]:
ftest(olscmax.model, olscmax_null.model)

F-test: 2 models fitted on 36 observations
───────────────────────────────────────────────────────────────
     DOF  ΔDOF     SSR    ΔSSR      R²      ΔR²      F*   p(>F)
───────────────────────────────────────────────────────────────
[1]   21        0.1023          0.9779                         
[2]   19    -2  0.1705  0.0682  0.9632  -0.0147  5.3315  0.0168
───────────────────────────────────────────────────────────────

In [9]:
ftest(olscmax_null.model, olscmax_form.model, olscmax.model)

F-test: 3 models fitted on 36 observations
───────────────────────────────────────────────────────────────
     DOF  ΔDOF     SSR     ΔSSR      R²     ΔR²      F*   p(>F)
───────────────────────────────────────────────────────────────
[1]   19        0.1705           0.9632                        
[2]   20     1  0.1477  -0.0228  0.9681  0.0049  2.6302  0.1232
[3]   21     1  0.1023  -0.0453  0.9779  0.0098  7.0905  0.0170
───────────────────────────────────────────────────────────────

In [11]:
ftest(olscmax_null.model, olscmax_seq.model)

LoadError: ArgumentError: F test is only valid for nested models

In [15]:
ftest(olscmax.model, olscmax_noseq.model)

LoadError: ArgumentError: F test is only valid for nested models